### Import statements

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt

random.seed(0)
np.random.seed(0)

### Reward Matrix<br>




(rather than classical prisoners' dilemma with punishments so that players want to maximize their value)<br>
--------------------Cooperate------Defect<br>
Cooperate-------(2, 2)-----------(0, 3)<br>
Defect-------------(3, 0)-----------(1, 1)

In [ ]:
rewards = np.array([[[2, 2], [0, 3]], [[3, 0], [1, 1]]])

# Player class algorithms

## Superclass of all player algorithms

In [ ]:
class Player():
  def __init__(self):
    self.num_rounds_round = 0
    self.rewards_round = []
    self.cumulative_reward_round = 0
    self.cumulative_rewards_round = []
    self.average_reward_round = 0
    self.average_rewards_round = []

    self.num_rounds_total = 0
    self.rewards_total = []
    self.cumulative_reward_total = 0
    self.cumulative_rewards_total = []
    self.average_reward_total = 0
    self.average_rewards_total = []

  def get_action(self):
    pass

  def receive_reward(self, rewards, row, col, player):
    self.num_rounds_round += 1
    self.num_rounds_total += 1
    reward = rewards[row, col, player]
    self.rewards_round.append(reward)
    self.rewards_total.append(reward)
    self.cumulative_reward_round += reward
    self.cumulative_reward_total += reward
    self.cumulative_rewards_round.append(self.cumulative_reward_round)
    self.cumulative_rewards_total.append(self.cumulative_reward_total)
    self.average_reward_round = self.cumulative_reward_round / self.num_rounds_round
    self.average_reward_total = self.cumulative_reward_total / self.num_rounds_total
    self.average_rewards_round.append(self.average_reward_round)
    self.average_rewards_total.append(self.average_reward_total)

  def get_name(self):
    pass

  def reset_round(self):
    self.num_rounds_round = 0
    self.rewards_round = []
    self.cumulative_reward_round = 0
    self.cumulative_rewards_round = []
    self.average_reward_round = 0
    self.average_rewards_round = []

## Player classes

### Nice player always cooperates

In [ ]:
class Nice(Player):
  def __init__(self):
    super().__init__()
  
  def get_action(self):
    return 0

  def get_name(self):
    return "Nice"

### Nasty player always defects

In [ ]:
class Nasty(Player):
  def __init__(self):
    super().__init__()
  
  def get_action(self):
    return 1

  def get_name(self):
    return "Nasty"

### Tit for Tat player initially cooperates, then mirrors the opponent's previous action

In [ ]:
class TitForTat(Player):
  def __init__(self):
    super().__init__()
    self.opponent_last_move = 0

  def get_action(self):
    return self.opponent_last_move

  def receive_reward(self, rewards, row, col, player):
    super().receive_reward(rewards, row, col, player)
    if player == 0:
      self.opponent_last_move = col
    else:
      self.opponent_last_move = row

  def get_name(self):
    return "TitForTat"

  def reset_round(self):
    super().reset_round()
    self.opponent_last_move = 0

### EpsilonNasty defects with probability epsilon and cooperates with probability 1-epsilon

In [ ]:
class EpsilonNasty(Player):
  def __init__(self, epsilon):
    super().__init__()
    self.epsilon = epsilon
  
  def get_action(self):
    if random.uniform(0, 1) < self.epsilon:
      return 1
    else:
      return 0

  def get_name(self):
    return "EpsilonNasty (" + str(100*self.epsilon) + "%)"

### Grim Trigger cooperates until the opponent defects, then it defects for the rest of the episode

In [ ]:
class GrimTrigger(Player):
  def __init__(self):
    super().__init__()
    self.action = 0

  def get_action(self):
    return self.action

  def receive_reward(self, rewards, row, col, player):
    super().receive_reward(rewards, row, col, player)
    if self.action == 0:
      if (player == 0 and col == 1) or (player == 1 and row == 1):
        self.action = 1

  def get_name(self):
    return "GrimTrigger"

  def reset_round(self):
    super().reset_round()
    self.action = 0

### TitForTat with Forgiveness is like Tit for Tat but cooperates with the opponent with probability epsilon after the opponent defects

In [ ]:
class TitForTatForgiveness(Player):
  def __init__(self, epsilon):
    super().__init__()
    self.opponent_last_move = 0
    self.epsilon = epsilon

  def get_action(self):
    if self.opponent_last_move == 0:
      return 0
    elif random.uniform(0, 1) < self.epsilon:
      return 0
    else:
      return 1

  def receive_reward(self, rewards, row, col, player):
    super().receive_reward(rewards, row, col, player)
    if player == 0:
      self.opponent_last_move = col
    else:
      self.opponent_last_move = row

  def get_name(self):
    return "TitForTatForgiveness (" + str(100*self.epsilon) + "%)"

  def reset_round(self):
    super().reset_round()
    self.opponent_last_move = 0

### Win-Stay, Lose-Switch starts cooperating, and switches strategies every time the opponent defects

In [ ]:
class WinStayLoseSwitch(Player):
  def __init__(self):
    super().__init__()
    self.action = 0

  def get_action(self):
    return self.action

  def receive_reward(self, rewards, row, col, player):
    super().receive_reward(rewards, row, col, player)
    if (player == 0 and col == 1) or (player == 1 and row == 1):
        self.action ^= 1

  def get_name(self):
    return "WinStayLoseSwitch"
  
  def reset_round(self):
    super().reset_round()
    self.action = 0

# Simulate function that takes in the players and outputs a graph with the average rewards for each player

In [ ]:
def simulate_match(player0, player1, num_iters, rewards, discount):
  iters = range(1, num_iters + 1)
  for i in iters:
    player0_action = player0.get_action()
    player1_action = player1.get_action()
    player0.receive_reward(rewards, player0_action, player1_action, 0)
    player1.receive_reward(rewards, player0_action, player1_action, 1)
    rewards *= discount
  
  name0 = player0.get_name()
  name1 = player1.get_name()
  name_num0 = name0
  name_num1 = name1
  if name0 == name1:
    name_num0 += "0"
    name_num1 += "1"

  plt.plot(iters, player0.average_rewards_round, 'b-', label = name_num0)
  plt.plot(iters, player1.average_rewards_round, 'r-', label = name_num1)
  plt.title("Average Rewards for " + name0 + " vs " + name1)
  plt.xlabel("Number of Iterations")
  plt.ylabel("Average Reward")
  plt.xlim(left = 1, right = num_iters + 1)
  plt.ylim(bottom = -0.1, top = (np.max(rewards) + 0.1))
  plt.legend()
  plt.show()

## Simulation Results

In [ ]:
player0 = Nice()
player1 = Nice()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = Nasty()
player1 = Nasty()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = Nice()
player1 = Nasty()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = Nice()
player1 = TitForTat()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = Nasty()
player1 = TitForTat()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = TitForTat()
player1 = TitForTat()
simulate_match(player0, player1, 100, rewards, 1)

In [ ]:
player0 = EpsilonNasty(0.05)
player1 = TitForTat()
simulate_match(player0, player1, 100, rewards, 1)

# Tournament function to simulate a randomly matched tournament and rank players by their average cumulative points per game

In [ ]:
def tournament(array0, array1, num_rounds, num_iters, rewards, discount):
  for _ in range(num_rounds):
    rewards_copy = np.copy(rewards).astype('float32')
    np.random.shuffle(array0)
    np.random.shuffle(array1)
    array0[0].reset_round()
    array1[0].reset_round()
    for _ in range(num_iters):
      player0_action = array0[0].get_action()
      player1_action = array1[0].get_action()
      array0[0].receive_reward(rewards_copy, player0_action, player1_action, 0)
      array1[0].receive_reward(rewards_copy, player0_action, player1_action, 1)
      rewards_copy *= discount
    array0[0].reset_round()
    array1[0].reset_round()

  array_of_players = np.concatenate((array0, array1))
  scores = []
  for i in range(array_of_players.shape[0]):
    scores.append(array_of_players[i].average_reward_total)
  scores = np.array(scores)
  places = np.argsort(scores)
  results = np.take(array_of_players, np.flip(places), axis=0)
  return results

In [ ]:
nice0 = Nice()
nice1 = Nice()
nasty0 = Nasty()
nasty1 = Nasty()
titfortat0 = TitForTat()
titfortat1 = TitForTat()
epsilonnasty0 = EpsilonNasty(0.05)
epsilonnasty1 = EpsilonNasty(0.05)
grimtrigger0 = GrimTrigger()
grimtrigger1 = GrimTrigger()
titfortatforgiveness0 = TitForTatForgiveness(0.05)
titfortatforgiveness1 = TitForTatForgiveness(0.05)
winstayloseswitch0 = WinStayLoseSwitch()
winstayloseswitch1 = WinStayLoseSwitch()

array0 = []
array1 = []
array0.append(nice0)
array1.append(nice1)
array0.append(nasty0)
array1.append(nasty1)
array0.append(titfortat0)
array1.append(titfortat1)
array0.append(epsilonnasty0)
array1.append(epsilonnasty1)
array0.append(grimtrigger0)
array1.append(grimtrigger1)
array0.append(titfortatforgiveness0)
array1.append(titfortatforgiveness1)
array0.append(winstayloseswitch0)
array1.append(winstayloseswitch1)

array0 = np.array(array0)
array1 = np.array(array1)

num_rounds = 10000
num_iters = 1000
discount = 1
results = tournament(array0, array1, num_rounds, num_iters, rewards, discount)

print("Tournament Results")
print(str(num_rounds) + " rounds, " + str(num_iters) + " iterations, discount = " + str(discount) + "\n")
for i in range(results.shape[0]):
  print(results[i].get_name() + " " + str(results[i].average_reward_total))

In [ ]:
def plot_averages(player0, player1, min_y, max_y, num_rounds, num_iters, discount):
  num_rounds0 = player0.num_rounds_total
  num_rounds1 = player1.num_rounds_total
  rounds0 = range(1, num_rounds0 + 1)
  rounds1 = range(1, num_rounds1 + 1)

  name0 = player0.get_name()
  name1 = player1.get_name()
  name_num0 = name0
  name_num1 = name1
  if name0 == name1:
    name_num0 += "0"
    name_num1 += "1"

  plt.plot(rounds0, player0.average_rewards_total, 'b-', label = name_num0)
  plt.plot(rounds1, player1.average_rewards_total, 'r-', label = name_num1)
  plt.title("Average Rewards for " + name0 + " vs " + name1 + "\n" + str(num_rounds) + " rounds, " + str(num_iters) + " iterations, discount = " + str(discount))
  plt.xlabel("Number of Iterations")
  plt.ylabel("Average Reward")
  plt.xlim(left = 1, right = (max(num_rounds0, num_rounds1) + 1))
  plt.ylim(bottom = min_y, top = max_y)
  plt.legend()
  plt.show()

## Show convergence for each player in tournament

In [ ]:
y_min = 1
y_max = 2

In [ ]:
plot_averages(nice0, nice1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(nasty0, nasty1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(titfortat0, titfortat1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(epsilonnasty0, epsilonnasty1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(grimtrigger0, grimtrigger1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(titfortatforgiveness0, titfortatforgiveness1, y_min, y_max, num_rounds, num_iters, discount)

In [ ]:
plot_averages(winstayloseswitch0, winstayloseswitch1, y_min, y_max, num_rounds, num_iters, discount)